In [49]:
%%bash
k3d cluster create

INFO[0000] Prep: Network                                
INFO[0000] Created network 'k3d-k3s-default'            
INFO[0000] Created image volume k3d-k3s-default-images  
INFO[0000] Starting new tools node...                   
INFO[0000] Starting node 'k3d-k3s-default-tools'        
INFO[0001] Creating node 'k3d-k3s-default-server-0'     
INFO[0001] Creating LoadBalancer 'k3d-k3s-default-serverlb' 
INFO[0001] Using the k3d-tools node to gather environment information 
INFO[0001] HostIP: using network gateway 172.29.0.1 address 
INFO[0001] Starting cluster 'k3s-default'               
INFO[0001] Starting servers...                          
INFO[0001] Starting node 'k3d-k3s-default-server-0'     
INFO[0008] All agents already running.                  
INFO[0008] Starting helpers...                          
INFO[0008] Starting node 'k3d-k3s-default-serverlb'     
INFO[0015] Injecting records for hostAliases (incl. host.k3d.internal) and for 2 network members into CoreDNS configmap... 

In [50]:
%%bash
helm repo add crossplane-stable https://charts.crossplane.io/stable
helm repo update
helm install crossplane --namespace crossplane-system --create-namespace crossplane-stable/crossplane 

"crossplane-stable" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "localstack-repo" chart repository
...Successfully got an update from the "cilium" chart repository
...Successfully got an update from the "crossplane-stable" chart repository
...Successfully got an update from the "metallb" chart repository
...Successfully got an update from the "prometheus-community" chart repository
Update Complete. ⎈Happy Helming!⎈
NAME: crossplane
LAST DEPLOYED: Thu Jan  2 16:42:28 2025
NAMESPACE: crossplane-system
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Release: crossplane

Chart Name: crossplane
Chart Description: Crossplane is an open source Kubernetes add-on that enables platform teams to assemble infrastructure from multiple vendors, and expose higher level self-service APIs for application teams to consume.
Chart Version: 1.18.2
Chart Application Version: 1.18.2

Kube 

### Wait until all pods are running before proceeding:

In [56]:
%%bash
kubectl get pods -n crossplane-system

NAME                                    READY   STATUS    RESTARTS   AGE
crossplane-5cdc8c8bc7-847cz             0/1     Running   0          22s
crossplane-rbac-manager-c79f85f-v9t5t   1/1     Running   0          22s


In [57]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: pkg.crossplane.io/v1beta1
kind: Function
metadata:
  name: function-patch-and-transform
spec:
  package: xpkg.upbound.io/crossplane-contrib/function-patch-and-transform:v0.1.4
EOF

function.pkg.crossplane.io/function-patch-and-transform created


In [58]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: pkg.crossplane.io/v1
kind: Provider
metadata:
  name: provider-kubernetes
spec:
  package: xpkg.upbound.io/upbound/provider-kubernetes:v0.16.0
  runtimeConfigRef:
    apiVersion: pkg.crossplane.io/v1beta1
    kind: DeploymentRuntimeConfig
    name: provider-kubernetes
---
apiVersion: pkg.crossplane.io/v1beta1
kind: DeploymentRuntimeConfig
metadata:
  name: provider-kubernetes
spec:
  serviceAccountTemplate:
    metadata:
      name: provider-kubernetes
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: provider-kubernetes-cluster-admin
subjects:
  - kind: ServiceAccount
    name: provider-kubernetes
    namespace: crossplane-system
roleRef:
  kind: ClusterRole
  name: cluster-admin
  apiGroup: rbac.authorization.k8s.io
EOF

provider.pkg.crossplane.io/provider-kubernetes created
deploymentruntimeconfig.pkg.crossplane.io/provider-kubernetes created
clusterrolebinding.rbac.authorization.k8s.io/provider-kubernetes-cluster-admin created


In [59]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: kubernetes.crossplane.io/v1alpha1
kind: ProviderConfig
metadata:
  name: default
spec:
  credentials:
    source: InjectedIdentity
EOF

providerconfig.kubernetes.crossplane.io/default created


The cluster and Crossplane are now set up!

# Reproduction Steps

In [60]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: CompositeResourceDefinition
metadata:
  name: xsimples.example.guidewire.net
spec:
  group: example.guidewire.net
  names:
    kind: xSimple
    plural: xsimples
  claimNames:
    kind: Simple
    plural: simples
  versions:
  - name: v1alpha1
    served: true
    referenceable: true
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
            required:
              - foo
EOF

compositeresourcedefinition.apiextensions.crossplane.io/xsimples.example.guidewire.net created


In [61]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: simple-composition-fn
spec:
  mode: Pipeline
  compositeTypeRef:
    apiVersion: example.guidewire.net/v1alpha1
    kind: xSimple
  pipeline:
  - step: patch-and-transform
    functionRef:
      name: function-patch-and-transform
    input:
      apiVersion: pt.fn.crossplane.io/v1beta1
      kind: Resources
      resources:
      - name: themap
        base:
          apiVersion: kubernetes.crossplane.io/v1alpha2
          kind: Object
          spec:
            forProvider:
              manifest:
                apiVersion: v1
                kind: ConfigMap
                metadata:
                  namespace: default
                data:
                  constant: first version
                  foo: replace me
        patches:
          - type: FromCompositeFieldPath
            fromFieldPath: spec.foo
            toFieldPath: metadata.name
EOF

composition.apiextensions.crossplane.io/simple-composition-fn created


In [62]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test-infinity-ok
  namespace: default
spec:
  foo: ""
  compositionRef:
    name: simple-composition-fn
EOF

simple.example.guidewire.net/test-infinity-ok created


In [63]:
%%bash
kubectl get simple.example.guidewire.net
kubectl get configmap

NAME               SYNCED   READY   CONNECTION-SECRET   AGE
test-infinity-ok   True     True                        4s
NAME                           DATA   AGE
kube-root-ca.crt               1      2m5s
test-infinity-ok-bdcxh-zs2qq   2      3s


In [64]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: simple-composition-legacy
spec:
  compositeTypeRef:
    apiVersion: example.guidewire.net/v1alpha1
    kind: xSimple
  resources:
    - name: themap
      base:
        apiVersion: kubernetes.crossplane.io/v1alpha2
        kind: Object
        spec:
          forProvider:
            manifest:
              apiVersion: v1
              kind: ConfigMap
              metadata:
                namespace: default
              data:
                constant: first version
                foo: replace me
      patches:
        - type: FromCompositeFieldPath
          fromFieldPath: spec.foo
          toFieldPath: metadata.name
EOF

composition.apiextensions.crossplane.io/simple-composition-legacy created


In [65]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test-infinity-bad
  namespace: default
spec:
  foo: ""
  compositionRef:
    name: simple-composition-legacy
EOF

simple.example.guidewire.net/test-infinity-bad created


In [66]:
%%bash
kubectl get simple.example.guidewire.net
kubectl get configmap
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l

NAME                SYNCED   READY   CONNECTION-SECRET   AGE
test-infinity-bad   True     False                       6s
test-infinity-ok    True     True                        25s
NAME                            DATA   AGE
kube-root-ca.crt                1      2m27s
test-infinity-bad-5htwh-2lszp   2      2s
test-infinity-bad-5htwh-4fcsr   2      4s
test-infinity-bad-5htwh-64zd6   2      1s
test-infinity-bad-5htwh-6gd9k   2      2s
test-infinity-bad-5htwh-79ft4   2      1s
test-infinity-bad-5htwh-8dtkh   2      4s
test-infinity-bad-5htwh-8r4c8   2      4s
test-infinity-bad-5htwh-9n2hv   2      1s
test-infinity-bad-5htwh-bp6xg   2      5s
test-infinity-bad-5htwh-cvrgn   2      5s
test-infinity-bad-5htwh-d6dvp   2      2s
test-infinity-bad-5htwh-dnq9d   2      6s
test-infinity-bad-5htwh-dznj5   2      2s
test-infinity-bad-5htwh-flqnv   2      5s
test-infinity-bad-5htwh-fls6g   2      3s
test-infinity-bad-5htwh-gz9hs   2      3s
test-infinity-bad-5htwh-j66ll   2      5s
test-infinity-ba

In [67]:
%%bash
kubectl delete simple.example.guidewire.net test-infinity-bad
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l
sleep 1 ; date ; kubectl get configmap | wc -l

simple.example.guidewire.net "test-infinity-bad" deleted
Thu Jan  2 16:45:03 EST 2025
      88
Thu Jan  2 16:45:04 EST 2025
      57
Thu Jan  2 16:45:05 EST 2025
      31
Thu Jan  2 16:45:07 EST 2025
       3
Thu Jan  2 16:45:08 EST 2025
       3
